In [1]:
import symforce
symforce.set_epsilon_to_symbol()
from symforce.values import Values
import numpy as np
import cv2 as cv
import symforce.symbolic as sf

In [2]:
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer

In [3]:
# make the sampled data: 
ar = 1.0
br = 2.0
cr = 1.0
ae = 2.0
be = -1.0
ce = 5.0
N = 100
w_sigma = 1.0
inv_sigma = 1.0 / w_sigma

rng = np.random.default_rng()

x_data = np.linspace(0, 1, N)
y_data = np.exp(ar * x_data * x_data + br * x_data + cr) + rng.normal(0, w_sigma, N)

In [4]:
def error(abc: sf.V3, x: sf.Scalar, y: sf.Scalar) -> sf.V1:
    return sf.V1(y - sf.exp(abc[0]*x*x + abc[1]*x + abc[2]))


In [13]:
initial_values = Values(
    abc = sf.V3([ae, be, ce]),
    xdata = x_data.tolist(),
    ydata = y_data.tolist(),
    epsilon = sf.numeric_epsilon,
)

In [14]:
factors = []
for i in range(len(x_data)):
    factors.append(Factor(
        residual=error,
        keys=["abc", f"xdata[{i}]", f"ydata[{i}]"],
    ))

In [29]:
optimizer = Optimizer(
    factors=factors,
    optimized_keys=["abc"],
    debug_stats=True,
    params=Optimizer.Params(iterations=1000)
)
result = optimizer.optimize(initial_values)
result

Optimizer.Result(initial_values=Values(
  abc: [2.0]
[-1.0]
[5.0],
  xdata: [0.0, 0.010101010101010102, 0.020202020202020204, 0.030303030303030304, 0.04040404040404041, 0.05050505050505051, 0.06060606060606061, 0.07070707070707072, 0.08080808080808081, 0.09090909090909091, 0.10101010101010102, 0.11111111111111112, 0.12121212121212122, 0.13131313131313133, 0.14141414141414144, 0.15151515151515152, 0.16161616161616163, 0.17171717171717174, 0.18181818181818182, 0.19191919191919193, 0.20202020202020204, 0.21212121212121213, 0.22222222222222224, 0.23232323232323235, 0.24242424242424243, 0.25252525252525254, 0.26262626262626265, 0.27272727272727276, 0.2828282828282829, 0.29292929292929293, 0.30303030303030304, 0.31313131313131315, 0.32323232323232326, 0.33333333333333337, 0.3434343434343435, 0.3535353535353536, 0.36363636363636365, 0.37373737373737376, 0.38383838383838387, 0.393939393939394, 0.4040404040404041, 0.4141414141414142, 0.42424242424242425, 0.43434343434343436, 0.4444444444444445,

In [47]:
a, b, c = result.optimized_values.get('abc')
y_cal = np.exp(a * x_data * x_data + b * x_data + c)
y_gt = np.exp(ar * x_data * x_data + br * x_data + cr)

In [52]:
import matplotlib.pyplot as plt
%matplotlib qt
plt.plot(x_data, y_data, '.')
plt.plot(x_data, y_cal)
plt.plot(x_data, y_gt)
plt.show()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""
